In [15]:
from geopandas import gpd
from shapely.geometry import box, Polygon
import asf_search as asf
from datetime import date,datetime,timedelta
import json
import os
from tqdm import tqdm

### Get Image Acquisition

In [4]:
data_kabkota=gpd.read_file('/data/raw/kab_indo_ST23.gpkg')
for i in tqdm(data_kabkota.idkab.unique()):
    if not os.path.exists(f'/data/ksa/01_Image_Acquisition/06_SatelliteCoverageKabkota/{i}_metadata_ASF.json'):
        kabkota_=data_kabkota.query('idkab==@i')
        bounds = kabkota_.total_bounds
        gdf_bounds = gpd.GeoSeries([box(*bounds)])
        wkt_aoi = gdf_bounds.to_wkt().values.tolist()[0]
        results = asf.search(
            platform= asf.PLATFORM.SENTINEL1A,
            processingLevel=[asf.PRODUCT_TYPE.GRD_HD],
            start = date(2023, 1, 1),
            end = date(2023, 12, 31),
            intersectsWith = wkt_aoi
            )
        metadata = results.geojson()
        json_object = json.dumps(metadata)
        with open(f'/data/ksa/01_Image_Acquisition/06_SatelliteCoverageKabkota/{i}_metadata_ASF.json', 'w') as f:
            f.write(json_object)

100%|██████████| 517/517 [00:00<00:00, 172992.04it/s]


## Get the coverage

In [5]:
def read_json(idkabkot):
    path_file = f'/data/ksa/01_Image_Acquisition/06_SatelliteCoverageKabkota/{idkabkot}_metadata_ASF.json'
    with open(path_file, 'r') as file:
        data = json.load(file)
    return data
def write_json(idkabkot, dict_list):
    path_file = f'/data/ksa/01_Image_Acquisition/06_SatelliteCoverageKabkota/{idkabkot}_coverage_ASF.json'
    with open(path_file, 'w') as json_file:
        json.dump(dict_list, json_file, ensure_ascii=False, indent=4)
def get_df_asf(idkabkot):
    data = read_json(idkabkot)
    asf_images = []
    for item in data['features']:
        filename = item['properties']['fileName'][:-4]
        tahun = item['properties']['processingDate'][:4]
        bulan = item['properties']['processingDate'][5:7]
        tanggal = item['properties']['processingDate'][8:10]
        geometry = item['geometry']['coordinates'][0]
        polygon = Polygon(geometry)
        asf_images.append(
           {
               'filename': filename, 
               'tahun': tahun,
               'bulan': bulan,
               'tanggal': tanggal,
               'geometry': polygon
           }
        )
    return gpd.GeoDataFrame(asf_images, geometry='geometry')
def get_df_input(idkabkot):
    gdf_mgrs = gpd.read_file('/data/ksa/00_Data_Input/indo_mgrs_grid_100km_overlay.gpkg')
    gdf_kabkot = gpd.read_file('/data/raw/kab_indo_ST23.gpkg')
    gdf_kabkot = gdf_kabkot[gdf_kabkot['idkab'] == idkabkot]
    intersecting_gdf = gpd.sjoin(gdf_mgrs, gdf_kabkot, how='inner', predicate='intersects')
    gdf_mgrs = gdf_mgrs[gdf_mgrs.index.isin(intersecting_gdf.index)]
    return gdf_mgrs, gdf_kabkot
def generate_date_pairs(year):
    start_date = datetime(year, 1, 1)
    date_pairs = []
    while start_date.year == year:
        end_date = start_date + timedelta(days=11)
        # Ensure the end date is still within the same year
        if end_date.year != year:
            break
        date_pairs.append([start_date.strftime('%Y-%m-%d'), end_date.strftime('%Y-%m-%d')])
        start_date = end_date + timedelta(days=1)
    return date_pairs
def get_periode_data(row, periode, gdf_asf):
    startPeriode = periode[0].split('-')
    endPeriode = periode[1].split('-')
    periode = {
        'start_periode': periode[0],
        'end_periode': periode[1]
    }
    gdf_asf_filter = gdf_asf[
        ((gdf_asf['tahun'] >= startPeriode[0]) & (gdf_asf['bulan'] >= startPeriode[1]) & (gdf_asf['tanggal'] >= startPeriode[2])) &
        ((gdf_asf['tahun'] <= endPeriode[0]) & (gdf_asf['bulan'] <= endPeriode[1]) & (gdf_asf['tanggal'] <= endPeriode[2]))
    ]
    if gdf_asf_filter.empty:
        periode.update({
            'image_asf': '',
            'coverage': 0,
            'alasan': 'Tidak ada image periode tersebut'
        })
        return periode
    overlapping_rows = gdf_asf_filter[gdf_asf_filter.intersects(row['geometry'])]
    if overlapping_rows.empty:
        periode.update({
            'image_asf': '',
            'coverage': 0,
            'alasan': 'Tidak ada image yang overlap dengan MGRS'
        })
        return periode
    periode['image_asf'] = overlapping_rows['filename'].to_list()
    df_temp = overlapping_rows.dissolve()
    area1 = row['geometry'].area
    intersection = row['geometry'].intersection(df_temp['geometry'])[0].area
    percentage_covered = (intersection / area1) * 100
    periode['coverage'] = percentage_covered
    periode['alasan'] = 'Coverage di bawah 80%' if percentage_covered < 80 else 'Sudah OK!'
    return periode
def process_rows(gdf_mgrs, gdf_asf):
    dict_list = []
    periode_range = []
    years = [2021, 2022, 2023]
    for year in years:
        date_pairs = generate_date_pairs(year)
        date_pairs[-1][1] = date_pairs[-1][1][:-2] + '31'
        periode_range = periode_range + date_pairs  
    for _, row in gdf_mgrs.iterrows():
        periode_list = [get_periode_data(row, periode, gdf_asf) for periode in periode_range]
        dict_list.append({
            'id': row['MGRS'],
            'geometry': row['geometry'].wkt,
            'periode': periode_list
        })
    return dict_list

In [14]:
for i in tqdm(data_kabkota.idkab.unique()):
    if not os.path.exists(f'/data/ksa/01_Image_Acquisition/06_SatelliteCoverageKabkota/{i}_coverage_ASF.json'):
        gdf_mgrs, gdf_prov = get_df_input(i)
        gdf_asf = get_df_asf(i)
        dict_list = process_rows(gdf_mgrs, gdf_asf)
        write_json(i, dict_list)

100%|██████████| 517/517 [00:00<00:00, 221655.44it/s]


## Calculate the precentage

In [19]:
with open('/data/ksa/01_Image_Acquisition/06_SatelliteCoverageKabkota/3507_coverage_ASF.json') as file:
    y=json.load(file)

In [26]:
y[0]['periode'][60]

{'start_periode': '2023-01-01',
 'end_periode': '2023-01-12',
 'image_asf': ['S1A_IW_GRDH_1SDV_20230112T105819_20230112T105849_046749_059AC9_5348'],
 'coverage': 69.6608290481277,
 'alasan': 'Coverage di bawah 80%'}